In [11]:
import os 
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
from PIL import Image
import base64
import io

In [9]:
####### Upload Background Remove Model Into S3 Bucket First########
####### This should be done once at a time since bucket will remain there unless deleted #########
import tarfile
sess = boto3.client('s3')
bucket_name = 'mybackgroundremoval'
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    tar.add('u2net.pth')
sess.upload_file(
    'model.tar.gz',
    bucket_name,
    'u2net/model.tar.gz'
)

[01/21/25 10:50:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=80436;file:///Users/msd/Desktop/MLOps/venv/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=753108;file:///Users/msd/Desktop/MLOps/venv/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

In [17]:
def deploy_model_to_sagemaker(
    model_data_path, 
    role = None,
    instance_type = 'ml.t2.medium',
    initial_instance_account = 1,
    endpoint_name = 'custom-model-endpoint',
    framework_version= '1.8.1',
    py_version='py3'
):
    if role is None:
        raise ValueError("no role applied for sagemaker")

    model = PyTorchModel(
        model_data=model_data_path,
        role=role,
        framework_version=framework_version,
        py_version=py_version,
        entry_point="inference.py",
        source_dir = 'code'
    )

    predictor = model.deploy(
        initial_instance_count=initial_instance_account,
        instance_type=instance_type,
        endpoint_name=endpoint_name
    )

    return predictor

In [87]:
def create_instance_script():
    return """
import json
import io
from PIL import Image
import base64
import os
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class REBNCONV(nn.Module):
    def __init__(self,in_ch=3,out_ch=3,dirate=1):
        super(REBNCONV,self).__init__()

        self.conv_s1 = nn.Conv2d(in_ch,out_ch,3,padding=1*dirate,dilation=1*dirate)
        self.bn_s1 = nn.BatchNorm2d(out_ch)
        self.relu_s1 = nn.ReLU(inplace=True)

    def forward(self,x):

        hx = x
        xout = self.relu_s1(self.bn_s1(self.conv_s1(hx)))

        return xout

## upsample tensor 'src' to have the same spatial size with tensor 'tar'
def _upsample_like(src,tar):

    src = F.upsample(src,size=tar.shape[2:],mode='bilinear')

    return src


### RSU-7 ###
class RSU7(nn.Module):#UNet07DRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU7,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool3 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool4 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv5 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool5 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv6 = REBNCONV(mid_ch,mid_ch,dirate=1)

        self.rebnconv7 = REBNCONV(mid_ch,mid_ch,dirate=2)

        self.rebnconv6d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv5d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv4d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x
        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)
        hx = self.pool4(hx4)

        hx5 = self.rebnconv5(hx)
        hx = self.pool5(hx5)

        hx6 = self.rebnconv6(hx)

        hx7 = self.rebnconv7(hx6)

        hx6d =  self.rebnconv6d(torch.cat((hx7,hx6),1))
        hx6dup = _upsample_like(hx6d,hx5)

        hx5d =  self.rebnconv5d(torch.cat((hx6dup,hx5),1))
        hx5dup = _upsample_like(hx5d,hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5dup,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup,hx1),1))

        return hx1d + hxin

### RSU-6 ###
class RSU6(nn.Module):#UNet06DRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU6,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool3 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool4 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv5 = REBNCONV(mid_ch,mid_ch,dirate=1)

        self.rebnconv6 = REBNCONV(mid_ch,mid_ch,dirate=2)

        self.rebnconv5d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv4d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)
        hx = self.pool4(hx4)

        hx5 = self.rebnconv5(hx)

        hx6 = self.rebnconv6(hx5)


        hx5d =  self.rebnconv5d(torch.cat((hx6,hx5),1))
        hx5dup = _upsample_like(hx5d,hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5dup,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup,hx1),1))

        return hx1d + hxin

### RSU-5 ###
class RSU5(nn.Module):#UNet05DRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU5,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool3 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=1)

        self.rebnconv5 = REBNCONV(mid_ch,mid_ch,dirate=2)

        self.rebnconv4d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)
        hx = self.pool3(hx3)

        hx4 = self.rebnconv4(hx)

        hx5 = self.rebnconv5(hx4)

        hx4d = self.rebnconv4d(torch.cat((hx5,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup,hx1),1))

        return hx1d + hxin

### RSU-4 ###
class RSU4(nn.Module):#UNet04DRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU4,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=1)
        self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=1)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=2)

        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=1)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx = self.pool1(hx1)

        hx2 = self.rebnconv2(hx)
        hx = self.pool2(hx2)

        hx3 = self.rebnconv3(hx)

        hx4 = self.rebnconv4(hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.rebnconv2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.rebnconv1d(torch.cat((hx2dup,hx1),1))

        return hx1d + hxin

### RSU-4F ###
class RSU4F(nn.Module):#UNet04FRES(nn.Module):

    def __init__(self, in_ch=3, mid_ch=12, out_ch=3):
        super(RSU4F,self).__init__()

        self.rebnconvin = REBNCONV(in_ch,out_ch,dirate=1)

        self.rebnconv1 = REBNCONV(out_ch,mid_ch,dirate=1)
        self.rebnconv2 = REBNCONV(mid_ch,mid_ch,dirate=2)
        self.rebnconv3 = REBNCONV(mid_ch,mid_ch,dirate=4)

        self.rebnconv4 = REBNCONV(mid_ch,mid_ch,dirate=8)

        self.rebnconv3d = REBNCONV(mid_ch*2,mid_ch,dirate=4)
        self.rebnconv2d = REBNCONV(mid_ch*2,mid_ch,dirate=2)
        self.rebnconv1d = REBNCONV(mid_ch*2,out_ch,dirate=1)

    def forward(self,x):

        hx = x

        hxin = self.rebnconvin(hx)

        hx1 = self.rebnconv1(hxin)
        hx2 = self.rebnconv2(hx1)
        hx3 = self.rebnconv3(hx2)

        hx4 = self.rebnconv4(hx3)

        hx3d = self.rebnconv3d(torch.cat((hx4,hx3),1))
        hx2d = self.rebnconv2d(torch.cat((hx3d,hx2),1))
        hx1d = self.rebnconv1d(torch.cat((hx2d,hx1),1))

        return hx1d + hxin


##### U^2-Net ####
class U2NET(nn.Module):

    def __init__(self,in_ch=3,out_ch=1):
        super(U2NET,self).__init__()

        self.stage1 = RSU7(in_ch,32,64)
        self.pool12 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage2 = RSU6(64,32,128)
        self.pool23 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage3 = RSU5(128,64,256)
        self.pool34 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage4 = RSU4(256,128,512)
        self.pool45 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage5 = RSU4F(512,256,512)
        self.pool56 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage6 = RSU4F(512,256,512)

        # decoder
        self.stage5d = RSU4F(1024,256,512)
        self.stage4d = RSU4(1024,128,256)
        self.stage3d = RSU5(512,64,128)
        self.stage2d = RSU6(256,32,64)
        self.stage1d = RSU7(128,16,64)

        self.side1 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side2 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side3 = nn.Conv2d(128,out_ch,3,padding=1)
        self.side4 = nn.Conv2d(256,out_ch,3,padding=1)
        self.side5 = nn.Conv2d(512,out_ch,3,padding=1)
        self.side6 = nn.Conv2d(512,out_ch,3,padding=1)

        self.outconv = nn.Conv2d(6*out_ch,out_ch,1)

    def forward(self,x):

        hx = x

        #stage 1
        hx1 = self.stage1(hx)
        hx = self.pool12(hx1)

        #stage 2
        hx2 = self.stage2(hx)
        hx = self.pool23(hx2)

        #stage 3
        hx3 = self.stage3(hx)
        hx = self.pool34(hx3)

        #stage 4
        hx4 = self.stage4(hx)
        hx = self.pool45(hx4)

        #stage 5
        hx5 = self.stage5(hx)
        hx = self.pool56(hx5)

        #stage 6
        hx6 = self.stage6(hx)
        hx6up = _upsample_like(hx6,hx5)

        #-------------------- decoder --------------------
        hx5d = self.stage5d(torch.cat((hx6up,hx5),1))
        hx5dup = _upsample_like(hx5d,hx4)

        hx4d = self.stage4d(torch.cat((hx5dup,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.stage3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.stage2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.stage1d(torch.cat((hx2dup,hx1),1))


        #side output
        d1 = self.side1(hx1d)

        d2 = self.side2(hx2d)
        d2 = _upsample_like(d2,d1)

        d3 = self.side3(hx3d)
        d3 = _upsample_like(d3,d1)

        d4 = self.side4(hx4d)
        d4 = _upsample_like(d4,d1)

        d5 = self.side5(hx5d)
        d5 = _upsample_like(d5,d1)

        d6 = self.side6(hx6)
        d6 = _upsample_like(d6,d1)

        d0 = self.outconv(torch.cat((d1,d2,d3,d4,d5,d6),1))

        return F.sigmoid(d0), F.sigmoid(d1), F.sigmoid(d2), F.sigmoid(d3), F.sigmoid(d4), F.sigmoid(d5), F.sigmoid(d6)

### U^2-Net small ###
class U2NETP(nn.Module):

    def __init__(self,in_ch=3,out_ch=1):
        super(U2NETP,self).__init__()

        self.stage1 = RSU7(in_ch,16,64)
        self.pool12 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage2 = RSU6(64,16,64)
        self.pool23 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage3 = RSU5(64,16,64)
        self.pool34 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage4 = RSU4(64,16,64)
        self.pool45 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage5 = RSU4F(64,16,64)
        self.pool56 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

        self.stage6 = RSU4F(64,16,64)

        # decoder
        self.stage5d = RSU4F(128,16,64)
        self.stage4d = RSU4(128,16,64)
        self.stage3d = RSU5(128,16,64)
        self.stage2d = RSU6(128,16,64)
        self.stage1d = RSU7(128,16,64)

        self.side1 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side2 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side3 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side4 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side5 = nn.Conv2d(64,out_ch,3,padding=1)
        self.side6 = nn.Conv2d(64,out_ch,3,padding=1)

        self.outconv = nn.Conv2d(6*out_ch,out_ch,1)

    def forward(self,x):

        hx = x

        #stage 1
        hx1 = self.stage1(hx)
        hx = self.pool12(hx1)

        #stage 2
        hx2 = self.stage2(hx)
        hx = self.pool23(hx2)

        #stage 3
        hx3 = self.stage3(hx)
        hx = self.pool34(hx3)

        #stage 4
        hx4 = self.stage4(hx)
        hx = self.pool45(hx4)

        #stage 5
        hx5 = self.stage5(hx)
        hx = self.pool56(hx5)

        #stage 6
        hx6 = self.stage6(hx)
        hx6up = _upsample_like(hx6,hx5)

        #decoder
        hx5d = self.stage5d(torch.cat((hx6up,hx5),1))
        hx5dup = _upsample_like(hx5d,hx4)

        hx4d = self.stage4d(torch.cat((hx5dup,hx4),1))
        hx4dup = _upsample_like(hx4d,hx3)

        hx3d = self.stage3d(torch.cat((hx4dup,hx3),1))
        hx3dup = _upsample_like(hx3d,hx2)

        hx2d = self.stage2d(torch.cat((hx3dup,hx2),1))
        hx2dup = _upsample_like(hx2d,hx1)

        hx1d = self.stage1d(torch.cat((hx2dup,hx1),1))


        #side output
        d1 = self.side1(hx1d)

        d2 = self.side2(hx2d)
        d2 = _upsample_like(d2,d1)

        d3 = self.side3(hx3d)
        d3 = _upsample_like(d3,d1)

        d4 = self.side4(hx4d)
        d4 = _upsample_like(d4,d1)

        d5 = self.side5(hx5d)
        d5 = _upsample_like(d5,d1)

        d6 = self.side6(hx6)
        d6 = _upsample_like(d6,d1)

        d0 = self.outconv(torch.cat((d1,d2,d3,d4,d5,d6),1))

        return F.sigmoid(d0), F.sigmoid(d1), F.sigmoid(d2), F.sigmoid(d3), F.sigmoid(d4), F.sigmoid(d5), F.sigmoid(d6)

def model_fn(model_dir):
    device = torch.device("cude" if torch.cuda.is_available() else "cpu")
    model = U2NET()
    model.load_state_dict(torch.load(os.path.join(model_dir, 'u2net.pth'), map_location = device))
    model.eval()
    return model

def input_fn(request_body, request_count_instance):
    if request_count_instance == 'application/x-image':
        image = Image.open(io.BytesIO(request_body))
    elif request_count_instance == 'application/json':
        image_data = json.loads(request_body)['image']
        image = Image.open(io.BytesIO(base64.b64.decode(image_data)))
    else:
        raise ValueError("Unsupported Content Type: {request_content_type}")

    if image.mode != 'RGB':
        image = image.convert('RGB')

    transform = transforms.Compose([
                transforms.ToTensor(),
        ])
    im = transform(image)
    im = im.unsqueeze(0)
    return im

def predict_fn(input_data, model):
    with torch.no_grad():
        prediction = model(input_data)
        return prediction

def output_fn(prediction, response_content_type):
    out_image = prediction[0].squeeze()
    out_image = out_image.cpu().numpy()

    out_image = (out_image - out_image.min()) / (out_image.max() - out_image.min())  # Normalize to [0, 1]
    out_image = (out_image * 255).astype(np.uint8)  # Scale to [0, 255]

    out_image = Image.fromarray(out_image)

    # if len(out_image.shape) == 3:
    #     out_image = out_image.permute(1,2,0)

    # Convert to desired output format
    if response_content_type == 'application/x-image':
        buffer = io.BytesIO()
        out_image.save(buffer, format='PNG')
        return buffer.getvalue()
    elif response_content_type == 'application/json':
        buffer = io.BytesIO()
        out_image.save(buffer, format='PNG')
        image_data = base64.b64encode(buffer.getvalue()).decode('utf-8')
        return json.dumps({'image': image_data})
    else:
        raise ValueError(f"Unsupported content type: {response_content_type}")
"""

In [20]:
def main():
    os.makedirs('code', exist_ok=True)
    with open('code/inference.py', 'w') as p:
        p.write(create_instance_script())

    predictor = deploy_model_to_sagemaker(
        model_data_path='s3://mybackgroundremoval/u2net/model.tar.gz',
        endpoint_name='custom-endpoint-model'
    )    

    print("model deployed successfully ")

In [89]:
from botocore.config import Config
def preprocess_image(image_path ):
    image = Image.open(image_path).resize((320, 320))
    return image

def call_sagemaker_endpoint(image_path, endpoint_name):
    runtime = boto3.client('sagemaker-runtime', region_name = 'us-east-1', config=Config(read_timeout=220))
    image = preprocess_image(image_path)
    img_buffer = io.BytesIO()
    image.save(img_buffer, format='PNG')
    img_bytes = img_buffer.getvalue()

    response = runtime.invoke_endpoint(
        EndpointName = endpoint_name,
        ContentType = 'application/x-image',
        Body = img_bytes
    )

    output = response['Body'].read()
    return output

In [92]:
import json
if __name__ == "__main__":
    # main()
    response = call_sagemaker_endpoint("/Users/msd/Desktop/MLOps/zidane.png", "custom-endpoint-model")
    response_dict = json.loads(response)
    image_data_base64 = response_dict['image']
    image_data_bytes = base64.b64decode(image_data_base64)
    image = Image.open(io.BytesIO(image_data_bytes))
    image.save("BG_output.png")

In [95]:
###### In the end delete S3 bucket and Sagemaker resources either manually or through writing code below
from botocore.exceptions import ClientError
# Create boto3 clients
sagemaker_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')

# Function to delete all SageMaker models
def delete_sagemaker_models():
    models = sagemaker_client.list_models()['Models']
    for model in models:
        model_name = model['ModelName']
        print(f"Deleting SageMaker model: {model_name}")
        sagemaker_client.delete_model(ModelName=model_name)

# Function to delete all SageMaker endpoints and configurations
def delete_sagemaker_endpoints():
    endpoints = sagemaker_client.list_endpoints()['Endpoints']
    for endpoint in endpoints:
        endpoint_name = endpoint['EndpointName']
        print(f"Deleting SageMaker endpoint: {endpoint_name}")
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

    endpoint_configs = sagemaker_client.list_endpoint_configs()['EndpointConfigs']
    for config in endpoint_configs:
        config_name = config['EndpointConfigName']
        print(f"Deleting SageMaker endpoint config: {config_name}")
        sagemaker_client.delete_endpoint_config(EndpointConfigName=config_name)

# Function to delete all SageMaker training jobs
def delete_sagemaker_training_jobs():
    training_jobs = sagemaker_client.list_training_jobs()['TrainingJobSummaries']
    for job in training_jobs:
        job_name = job['TrainingJobName']
        try:
            # Get the status of the training job
            status = sagemaker_client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
            print(f"Training job '{job_name}' status: {status}")

            # Stop the job if it's still in progress
            if status == 'InProgress':
                print(f"Stopping training job: {job_name}")
                sagemaker_client.stop_training_job(TrainingJobName=job_name)
            elif status in ['Completed', 'Stopped', 'Failed']:
                print(f"Training job '{job_name}' has already been {status}. No need to stop.")
        except ClientError as e:
            print(f"Error while processing training job '{job_name}': {e}")


# Function to delete all objects in S3 bucket
def delete_s3_bucket(bucket_name):
    try:
        # List all objects in the S3 bucket
        objects = s3_client.list_objects_v2(Bucket=bucket_name).get('Contents', [])
        
        # Delete all objects in the bucket
        for obj in objects:
            print(f"Deleting object {obj['Key']} from bucket {bucket_name}")
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])

        # Delete the bucket itself
        print(f"Deleting S3 bucket: {bucket_name}")
        s3_client.delete_bucket(Bucket=bucket_name)
    except s3_client.exceptions.NoSuchBucket:
        print(f"S3 bucket {bucket_name} does not exist or has already been deleted.")
        
# Function to list and delete all SageMaker and S3 resources
def delete_sagemaker_and_s3_resources(s3_bucket_name):
    # Delete SageMaker resources
    delete_sagemaker_models()
    delete_sagemaker_endpoints()
    delete_sagemaker_training_jobs()

    # Delete S3 bucket and its contents
    delete_s3_bucket(s3_bucket_name)

# Provide the name of the S3 bucket
s3_bucket_name = 'your-s3-bucket-name'

# Call the function to delete all SageMaker and S3 resources
delete_sagemaker_and_s3_resources(bucket_name)
print("Action Completed!!!")

Training job 'RF-custom-sklearn-2025-01-20-06-41-18-040' status: Completed
Training job 'RF-custom-sklearn-2025-01-20-06-41-18-040' has already been Completed. No need to stop.
Training job 'RF-custom-sklearn-2025-01-19-15-07-12-029' status: Completed
Training job 'RF-custom-sklearn-2025-01-19-15-07-12-029' has already been Completed. No need to stop.
Training job 'RF-custom-sklearn-2025-01-19-14-57-46-394' status: Failed
Training job 'RF-custom-sklearn-2025-01-19-14-57-46-394' has already been Failed. No need to stop.
Training job 'RF-custom-sklearn-2025-01-19-14-53-08-746' status: Failed
Training job 'RF-custom-sklearn-2025-01-19-14-53-08-746' has already been Failed. No need to stop.
Training job 'RF-custom-sklearn-2025-01-19-14-27-36-797' status: Failed
Training job 'RF-custom-sklearn-2025-01-19-14-27-36-797' has already been Failed. No need to stop.
S3 bucket mybackgroundremoval does not exist or has already been deleted.
Action Completed!!!
